In [1]:
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [2]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [3]:
dataset.drop('User ID', axis=1)


,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
dataset['Purchased'].value_counts()


Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [6]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [7]:
dep=dataset[['Purchased']]
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep,test_size=0.30,random_state=0)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
param_grid={'alpha':[0.1, 0.5, 1.0, 1.5, 2.0],'force_alpha':[True,False],'fit_prior':[True,False]}
grid=GridSearchCV(BernoulliNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [10]:
Y_pred=grid.predict(X_test)
Y_pred

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1], dtype=int64)

In [11]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
print(cm)

[[37 42]
 [18 23]]


In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,Y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.67      0.47      0.55        79
           1       0.35      0.56      0.43        41

    accuracy                           0.50       120
   macro avg       0.51      0.51      0.49       120
weighted avg       0.56      0.50      0.51       120



In [13]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,Y_pred, average='macro')
f1_macro

0.49310053506054635

In [14]:
from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print("roc_auc score for the best parameter {} {:.2f}".format(grid.best_params_,roc_score ))

roc_auc score for the best parameter {'alpha': 0.5, 'fit_prior': False, 'force_alpha': True} 0.51


In [15]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013226,0.001965,0.029643,0.006208,0.1,True,True,"{'alpha': 0.1, 'fit_prior': True, 'force_alpha...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,9
1,0.015691,0.007666,0.016889,0.007243,0.1,True,False,"{'alpha': 0.1, 'fit_prior': True, 'force_alpha...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,9
2,0.008143,0.007882,0.014262,0.006096,0.1,False,True,"{'alpha': 0.1, 'fit_prior': False, 'force_alph...",0.469388,0.472527,0.527800,0.384921,0.560029,0.482933,0.059758,19
3,0.012922,0.007982,0.010026,0.000030,0.1,False,False,"{'alpha': 0.1, 'fit_prior': False, 'force_alph...",0.469388,0.472527,0.527800,0.384921,0.560029,0.482933,0.059758,19
4,0.009780,0.007195,0.016110,0.005260,0.5,True,True,"{'alpha': 0.5, 'fit_prior': True, 'force_alpha...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,9
5,0.008620,0.004726,0.014682,0.005660,0.5,True,False,"{'alpha': 0.5, 'fit_prior': True, 'force_alpha...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,9
6,0.011876,0.008229,0.019028,0.005267,0.5,False,True,"{'alpha': 0.5, 'fit_prior': False, 'force_alph...",0.469388,0.472527,0.527800,0.480769,0.560029,0.502103,0.035821,1
7,0.010271,0.004635,0.016336,0.007837,0.5,False,False,"{'alpha': 0.5, 'fit_prior': False, 'force_alph...",0.469388,0.472527,0.527800,0.480769,0.560029,0.502103,0.035821,1
8,0.012628,0.007004,0.017481,0.003421,1.0,True,True,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,9
9,0.012460,0.001500,0.015892,0.004699,1.0,True,False,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,9
